In [2]:
import pandas as pd
import re
import numpy as np
import PyPDF2
import requests
import io
from PyPDF2 import PdfFileReader

In [3]:
url = 'https://www.nkeconwatch.com/nk-uploads/KJU-speech-2013.pdf'

r = requests.get(url)
f = io.BytesIO(r.content)

reader = PdfFileReader(f)
contents = reader.getPage(0).extractText().split('\n')
contents

['New Year Address Made by Kim Jong Un ',
 'Pyongyang, January 1 (KCNA) -- The dear respected K im Jong Un made a New Year address on January 1, ',
 '2013. The following is its full text: ',
 'Dear comrades, ',
 "Officers and men of the heroic People's Army and al l the beloved fellow countrymen, ",
 'Dear compatriots, ',
 'Having seen out the year 2012 characterized by the events that will go down in the history of the nati on, ',
 'we are seeing in the new year 2013 full of high hop e and confidence in final victory. ',
 'Reflecting the unanimous reverence of all the servi ce personnel and people for President Kim Il Sung ',
 'and General Kim Jong Il, the fathers of the nation and eternal Sun of Juche, I would like first to pay  the ',
 'highest tribute and New Year greetings to them. ',
 'I extend my warm greetings to the service personnel  and people who are devotedly striving for the ',
 "prosperity of the country, firmly united around the  Workers' Party of Korea, and sincerely w

In [11]:
'''
convert list to string
'''

def listToString(s):
 
    # initialize an empty string
    str1 = ""
 
    # traverse in the string
    for ele in s:
        str1 += ele
 
    # return string
    return str1

speech = listToString(contents)
speech

"New Year Address Made by Kim Jong Un Pyongyang, January 1 (KCNA) -- The dear respected K im Jong Un made a New Year address on January 1, 2013. The following is its full text: Dear comrades, Officers and men of the heroic People's Army and al l the beloved fellow countrymen, Dear compatriots, Having seen out the year 2012 characterized by the events that will go down in the history of the nati on, we are seeing in the new year 2013 full of high hop e and confidence in final victory. Reflecting the unanimous reverence of all the servi ce personnel and people for President Kim Il Sung and General Kim Jong Il, the fathers of the nation and eternal Sun of Juche, I would like first to pay  the highest tribute and New Year greetings to them. I extend my warm greetings to the service personnel  and people who are devotedly striving for the prosperity of the country, firmly united around the  Workers' Party of Korea, and sincerely wish all th e families across the country harmony and greater 

In [15]:
'''
string to dataframe
'''
data = speech
kju_2013 = pd.DataFrame([x.split('.') for x in data.split('\n')])


In [13]:
kju_2013

,0,1,2,3,4,5,6,7,8,9,10,11
0,New Year Address Made by Kim Jong Un Pyongyang...,The following is its full text: Dear comrades...,Reflecting the unanimous reverence of all the...,I extend my warm greetings to the service per...,My New Year greetings also go to our compatri...,Last year was a historic year in which the gr...,The President and the General were great lead...,Last year we ce lebrated the centenary of the...,"By doing so, we have been able to glorify the...",Although they saw in 2012 in tears of blood a...,"With loyalty as pure as white gem, noble sen...",


In [16]:
print(kju_2013.transpose())

                                                    0
0   New Year Address Made by Kim Jong Un Pyongyang...
1    The following is its full text: Dear comrades...
2    Reflecting the unanimous reverence of all the...
3    I extend my warm greetings to the service per...
4    My New Year greetings also go to our compatri...
5    Last year was a historic year in which the gr...
6    The President and the General were great lead...
7    Last year we ce lebrated the centenary of the...
8    By doing so, we have been able to glorify the...
9    Although they saw in 2012 in tears of blood a...
10   With loyalty as pure as white gem, noble  sen...
11                                                   


In [4]:
kju1 = ''.join(c.lower() for c in contents if not c.isspace())
kju1

"new year address made by kim jong un pyongyang, january 1 (kcna) -- the dear respected k im jong un made a new year address on january 1, 2013. the following is its full text: dear comrades, officers and men of the heroic people's army and al l the beloved fellow countrymen, dear compatriots, having seen out the year 2012 characterized by the events that will go down in the history of the nati on, we are seeing in the new year 2013 full of high hop e and confidence in final victory. reflecting the unanimous reverence of all the servi ce personnel and people for president kim il sung and general kim jong il, the fathers of the nation and eternal sun of juche, i would like first to pay  the highest tribute and new year greetings to them. i extend my warm greetings to the service personnel  and people who are devotedly striving for the prosperity of the country, firmly united around the  workers' party of korea, and sincerely wish all th e families across the country harmony and greater 

In [5]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\gjrre\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True